#### Dependencies

In [1]:
import yt_dlp as youtube_dl
import os
import numpy as np
import pandas as pd
import requests
import multiprocessing
import os
import librosa
import time
from dotenv import load_dotenv

#### Global Constants

In [2]:
load_dotenv()
YT_API_KEY = os.getenv('YT_API_KEY')
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')
CPU_THREADS = multiprocessing.cpu_count()

#### Data

In [3]:
songs_data = pd.read_csv('data/songs_final.csv')

#### Functions

In [4]:
def is_youtube_video_available(video_id):
    url = f'https://www.googleapis.com/youtube/v3/videos?id={video_id}&key={YT_API_KEY}&part=status'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data['pageInfo']['totalResults'] > 0:
            return True  # Video is available
        else:
            return False  # Video not found or unavailable
    else:
        print(f"Error: Unable to check video status (HTTP {response.status_code})")
        return False

In [5]:
def is_song_downloaded(song_filename: str):
    files = set(entry.name for entry in os.scandir(DOWNLOAD_FOLDER) if entry.is_file())
    return song_filename in files

In [32]:
def download_song(args):
    video_id, video_index = args
    video_url = f"https://www.youtube.com/watch?v={video_id}"

    #if not is_youtube_video_available(video_id):
    #    print(f"ERROR: Video is not available: video_id={video_id}")
    #    return

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }],
        'outtmpl': os.path.join(DOWNLOAD_FOLDER, f'{video_index}^{video_id}^'+'%(title)s.%(ext)s'),
        'quiet': True,
        'no_warnings': True, 
        'verbose': False,
        'noplaylist': True,
        'nocheckcertificate': True,
        'ignoreerrors': True,
        'retries': 3,
        'continuedl': True,
        'max_filesize': 10 * 1024 * 1024,
    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        try:
            info_dict = ydl.extract_info(video_url, download=False)
            title = info_dict.get('title', None)
            song_filename = f"{video_index}^{video_id}^{title}.mp3"

            if is_song_downloaded(song_filename):
                return

            ydl.download([video_url])
        except Exception as e:
            print(f"YDL ERROR: {e} ", video_id)
            return

    song_path = os.path.join(DOWNLOAD_FOLDER, song_filename)        
    return song_path

In [30]:
def download_songs_parallel(songs_data, lower, upper):
    video_ids = songs_data["videoID"].to_numpy()[lower:upper]
    video_indices = list(range(lower, upper))

    args = list(zip(video_ids, video_indices))

    with multiprocessing.Pool(CPU_THREADS) as pool:
        results = pool.map(download_song, args)

In [33]:
download_songs_parallel(songs_data, 0, len(songs_data))

ERROR: [youtube] onNE83Q_frw: Video unavailable. This video is not available


YDL ERROR: 'NoneType' object has no attribute 'get'  onNE83Q_frw


ERROR: [youtube] hGc--_qdtBM: Video unavailable. This video is not available


YDL ERROR: 'NoneType' object has no attribute 'get'  hGc--_qdtBM


ERROR: [youtube] OKmAX-ss31o: Video unavailable. This video is not available


YDL ERROR: 'NoneType' object has no attribute 'get'  OKmAX-ss31o


ERROR: [youtube] 33ObS-mo4sE: Video unavailable. This video is not available


YDL ERROR: 'NoneType' object has no attribute 'get'  33ObS-mo4sE


KeyboardInterrupt: 